In [3]:
import numpy as np
import pandas as pd


In [16]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np



FileNotFoundError: [Errno 2] No such file or directory: 'time.csv'

In [5]:
# 데이터 로드
data = pd.read_csv("../crwaling/time.csv")
data = data.drop(columns=["Unnamed: 0"])

In [6]:
data

,노래제목,가수,가사,태그
0,Sunflower (Spider-Man: Into the Spider-Verse),Post Malone(포스트 말론),Ayy ayy ayy ayy Ooh ooh ooh ooh Ayy ayy Ooh oo...,morning
1,Casio,Jungle(정글),Casio Playing on my heart just like a casio Br...,morning
2,Late Night,Lucky Daye,The bigger we are the harder we fall We swingi...,morning
3,Warm On A Cold Night,HONNE(혼네),Okay its AM Youre tuned in with your main ma...,morning
4,How Does It Feel?,Samm Henshaw(샘 핸쇼우),How does it feel to be loved I just landed on ...,morning
...,...,...,...,...
3547,Love Like This,ZAYN(제인)CONNECT 아티스트,Oh I love it when you show when you show When ...,night
3548,Feel Better,Chymes,Verse You say that were fine But i leave feeli...,night
3549,Other Boys,Marshmello(마시멜로),Why cant you want me like the other boys do Th...,night
3550,Lies You Tell,Emotional Oranges,Still tryna find my way out Guess I dont have ...,night


In [7]:
# BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
# 감정 태그를 숫자로 매핑
tag_mapping = {tag: idx for idx, tag in enumerate(data['태그'].unique())}
inverse_tag_mapping = {v: k for k, v in tag_mapping.items()}
data['태그_encoded'] = data['태그'].map(tag_mapping)

# 데이터 분할
X_train,X_val, y_train,y_val = train_test_split(data["가사"],data['태그_encoded'], test_size=0.2, random_state=42)

In [9]:
import tensorflow as tf

In [10]:
tokenized_X_train = X_train.apply(lambda x: tokenizer.encode_plus(x, add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True))
tokenized_X_val = X_val.apply(lambda x: tokenizer.encode_plus(x, add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True))
max_length = 100  # Choose an appropriate maximum length

# Extract Token IDs, Segment IDs, and Attention Mask
token_ids_X_train = [seq['input_ids'] for seq in tokenized_X_train]
segment_ids_X_train = [seq['token_type_ids'] for seq in tokenized_X_train]
attention_mask_X_train = [seq['attention_mask'] for seq in tokenized_X_train]
token_ids_X_val = [seq['input_ids'] for seq in tokenized_X_val]
segment_ids_X_val = [seq['token_type_ids'] for seq in tokenized_X_val]
attention_mask_X_val = [seq['attention_mask'] for seq in tokenized_X_val]

# Pad sequences
padded_token_ids_X_train = tf.keras.preprocessing.sequence.pad_sequences(token_ids_X_train, maxlen=max_length)
padded_segment_ids_X_train = tf.keras.preprocessing.sequence.pad_sequences(segment_ids_X_train, maxlen=max_length)
padded_attention_mask_X_train = tf.keras.preprocessing.sequence.pad_sequences(attention_mask_X_train, maxlen=max_length)
padded_token_ids_X_val = tf.keras.preprocessing.sequence.pad_sequences(token_ids_X_val, maxlen=max_length)
padded_segment_ids_X_val = tf.keras.preprocessing.sequence.pad_sequences(segment_ids_X_val, maxlen=max_length)
padded_attention_mask_X_val = tf.keras.preprocessing.sequence.pad_sequences(attention_mask_X_val, maxlen=max_length)
def create_model(bert_model, max_length, num_classes):
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids")

    # Extract BERT output for [CLS] token
    bert_output = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)[0][:, 0, :]

    # Apply dropout
    dropout_output = tf.keras.layers.Dropout(0.2)(bert_output)

    # Output layer for your task
    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(dropout_output)

    # Define model inputs and outputs
    inputs = {"input_ids": input_ids, "attention_mask": attention_mask, "token_type_ids": token_type_ids}
    outputs = output_layer

    # Create and compile the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

    return model
model= create_model(bert_model,100,4)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [11]:
epochs = 1
batch_size = 16

history = model.fit(
    {"input_ids": padded_token_ids_X_train, "attention_mask": padded_attention_mask_X_train, "token_type_ids": padded_segment_ids_X_train},
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=({"input_ids": padded_token_ids_X_val, "attention_mask": padded_attention_mask_X_val, "token_type_ids": padded_segment_ids_X_val}, y_val)
)






178/178 [==============================] - 1391s 8s/step - loss: 1.4944 - accuracy: 0.2679 - val_loss: 1.3935 - val_accuracy: 0.2405


In [12]:
# 모델 저장
# Hugging Face Transformers 라이브러리를 사용하여 BERT 모델을 저장
bert_model.save_pretrained("bert_model_directory")
tokenizer.save_pretrained("bert_model_directory")
# 나머지 커스텀 모델을 저장
model.save("time.h5")

C:\Users\gjaischool\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
# BERT 모델 불러오기
bert_model_directory = "bert_model_directory2"
bert_model = TFBertModel.from_pretrained(bert_model_directory)
tokenizer = BertTokenizer.from_pretrained(bert_model_directory)

# 커스텀 모델 불러오기
custom_model_path = "custom_model2.h5"
custom_model = load_model(custom_model_path)

In [15]:
# 예측 결과를 원래 태그로 변환
predictions = model.predict(
    {"input_ids": padded_token_ids_X_val, "attention_mask": padded_attention_mask_X_val, "token_type_ids": padded_segment_ids_X_val})
predicted_tags = np.argmax(predictions, axis=1)
predicted_tags = [inverse_tag_mapping[tag] for tag in predicted_tags]

print("Predicted Tags:", predicted_tags)


23/23 [==============================] - 72s 3s/step
Predicted Tags: ['noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'noon', 'n